In [55]:
import pandas as pd
import pytest

from kedro_sales_forecast.pipelines.train.nodes import split_data, train_model, evaluate_model

In [2]:
def dummy_data():
    data =  pd.DataFrame({"unique_id": ["id_1", "id_1", "id_1", "id_1", "id_1"],
            "ds": ["2009-04-12", "2009-04-19", "2009-04-26", "2009-05-03", "2009-05-10"],
            "y": [0.613262, 0.442162, 0.307173, 0.059010, 0.113098],
            "Size": [98, 84, 66, 59, 78],
            "Type_A": [1, 0, 0, 1, 1],
            "Type_B": [0, 1, 1, 0, 0],
            "IsHoliday": [0, 1, 0, 0, 1],
            "Temperature": [45.539704, 30.207879, 37.307495, 24.212213, 24.291916],
            "Fuel_Price": [2.410365, 2.837114, 2.810372, 3.104646, 3.108914],
            "CPI": [3.010402, 6.039538, 5.034800, 6.648689, 3.530509],
            "Unemployment": [7.475086, 6.395689, 4.980317, 6.949863, 3.012275]}
            )
    data["ds"] = pd.to_datetime(data["ds"])
    return data

In [3]:
def dummy_parameters():
    parameters = {"model_options": {
                        'split_date': '2009-05-03',
                        'random_state': 0,
                        'n_estimators': 100,
                        'num_threads': 6,
                        'freqency': 'W',
                        'horizon': 2,
                        'lags': [1, 2, 4],
                        'date_features': ['week', 'month'],
                        'static_features': ['Size', 'Type_A', 'Type_B'],
                        'exogenous_features': ['IsHoliday', 'Temperature', 'Fuel_Price', 'Unemployment', 'CPI']
                        }
                 }
    return parameters

In [4]:
dummy_data = dummy_data()
dummy_parameters = dummy_parameters()

In [5]:
train_data, test_data = split_data(dummy_data, dummy_parameters["model_options"])
assert len(train_data) == 3
assert len(test_data) == 2

In [6]:
dummy_data

,unique_id,ds,y,Size,Type_A,Type_B,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment
0,id_1,2009-04-12,0.613262,98,1,0,0,45.539704,2.410365,3.010402,7.475086
1,id_1,2009-04-19,0.442162,84,0,1,1,30.207879,2.837114,6.039538,6.395689
2,id_1,2009-04-26,0.307173,66,0,1,0,37.307495,2.810372,5.034800,4.980317
3,id_1,2009-05-03,0.059010,59,1,0,0,24.212213,3.104646,6.648689,6.949863
4,id_1,2009-05-10,0.113098,78,1,0,1,24.291916,3.108914,3.530509,3.012275


In [7]:
train_data, _ = split_data(dummy_data, dummy_parameters["model_options"])

In [8]:
train_data

,unique_id,ds,y,Size,Type_A,Type_B,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment
0,id_1,2009-04-12,0.613262,98,1,0,0,45.539704,2.410365,3.010402,7.475086
1,id_1,2009-04-19,0.442162,84,0,1,1,30.207879,2.837114,6.039538,6.395689
2,id_1,2009-04-26,0.307173,66,0,1,0,37.307495,2.810372,5.034800,4.980317


In [9]:
train_data.shape

(3, 11)

In [10]:
train_data.shape[0]

3

In [11]:
model = train_model(train_data, dummy_parameters["model_options"])

[12/25/23 10:37:54] WARNING  /home/karl/miniconda3/envs/python_3.9_time/lib/python3.9/site-packages ]8;id=474165;file:///home/karl/miniconda3/envs/python_3.9_time/lib/python3.9/warnings.py\warnings.py]8;;\:]8;id=294465;file:///home/karl/miniconda3/envs/python_3.9_time/lib/python3.9/warnings.py#109\109]8;;\
                             /mlforecast/core.py:409: UserWarning: The following series were                       
                             dropped completely due to the transformations and features: ['id_1'].                 
                             These series won't show up if you use                                                 
                             `MLForecast.forecast_fitted_values()`.                                                
                             You can set `dropna=False` or use transformations that require less                   
                             samples to mitigate this                                                              
                               warnings.warn(                                                                      
                                                                                                                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model = train_model(train_data, dummy_parameters["model_options"])                           │
│   2                                                                                              │
│                                                                                                  │
│ /home/karl/forecasting_projects/kedro-sales-forecast/src/kedro_sales_forecast/pipelines/train/no │
│ des.py:69 in train_model                                                                         │
│                                                                                                  │
│    66 │   │   │   │   │   │   date_features=date_features,                                       │
│    67 │   │   │   │   │   │   num_threads=num_threads                                            │
│    68 │   │   │   │   │      )                                                                   │
│ ❱  69 │   model.fit(train_data, static_features=static_features)                                 │
│    70 │   return model                                                                           │
│    71                                                                                            │
│    72                                                                                            │
│                                                                                                  │
│ /home/karl/miniconda3/envs/python_3.9_time/lib/python3.9/site-packages/mlforecast/forecast.py:52 │
│ 1 in fit                                                                                         │
│                                                                                                  │
│   518 │   │   │   if as_numpy:                                                                   │
│   519 │   │   │   │   X = ufp.to_numpy(X)                                                        │
│   520 │   │   │   del prep                                                                       │
│ ❱ 521 │   │   self.fit_models(X, y)                                                              │
│   522 │   │   if fitted:                                                                         │
│   523 │   │   │   fitted_values = self._compute_fitted_values(                                   │
│   524 │   │   │   │   base=base,                                                                 │
│                                                                                                  │
│ /home/karl/miniconda3/envs/python_3.9_time/lib/python3.9/site-packages/mlforecast/forecast.py:29 │
│ 1 in fit_models                                                                                  │
│                                                                                                  │
│   288 │   │   │   │   │   yh = y[keep, col]                                                      │
│   289 │   │   │   │   │   self.models_[name].append(clone(model).fit(Xh, yh))                    │
│   290 │   │   │   else:                                                                          │
│ ❱ 291 │   │   │   │   self.models_[name] = clone(model).fit(X, y)                                │
│   292 │   │   return self                                                                        │
│   293 │                                                                                          │
│   294 │   def _conformity_scores(                                                                │
│                                                                                                  │
│ /home/karl/miniconda3/envs/python_3.9_time/lib/python3.9/si

In [ ]:
assert model.freq == 'W'
assert model.models["XGBRegressor"].n_estimators == 100

In [33]:
store_sales_weekly = pd.DataFrame(catalog.load('store_sales_weekly'))

[12/25/23 10:48:41] INFO     Loading data from store_sales_weekly (ParquetDataset)...           ]8;id=696431;file:///home/karl/miniconda3/envs/python_3.9_time/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=991395;file:///home/karl/miniconda3/envs/python_3.9_time/lib/python3.9/site-packages/kedro/io/data_catalog.py#482\482]8;;\

In [34]:
store_sales_weekly = store_sales_weekly.sort_values(by=['ds', 'unique_id'])
store_sales_weekly.head()

,unique_id,ds,y,Size,Type_A,Type_B,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment
0,id_7,2001-09-30,0.098760,84,0,1,0,35.678620,2.270796,5.321664,9.990810
1,id_7,2001-10-07,0.264158,84,0,1,0,24.870336,2.632353,4.597994,8.467220
2,id_7,2001-10-14,0.335845,84,0,1,0,42.804482,3.006857,4.401153,8.103670
3,id_7,2001-10-21,0.235161,84,0,1,0,21.197120,2.380959,8.386135,5.155749
4,id_7,2001-10-28,0.479848,84,0,1,0,38.324511,2.277695,5.638826,7.920773


In [35]:
store_sales_weekly.groupby('unique_id')['ds'].count()


unique_id
id_0    222
id_1     97
id_2    167
id_3    242
id_4    373
id_5    301
id_6    245
id_7    409
id_8     59
id_9    261
Name: ds, dtype: int64

In [36]:
store_sales_weekly_sample = store_sales_weekly.query(" unique_id in('id_1', 'id_8') ")

In [38]:
store_sales_weekly_sample.shape

(156, 11)

In [37]:
store_sales_weekly_sample.to_csv('../data/05_model_input/future_data.csv', encoding='utf-8', index=False)

In [39]:
store_sales_weekly_sample.to_dict()


{
    'unique_id': {
        1452: 'id_1',
        1461: 'id_1',
        1470: 'id_1',
        1479: 'id_1',
        1488: 'id_1',
        1497: 'id_1',
        1506: 'id_1',
        1515: 'id_1',
        1524: 'id_1',
        1533: 'id_1',
        1542: 'id_1',
        1551: 'id_1',
        1560: 'id_1',
        1569: 'id_1',
        1578: 'id_1',
        1587: 'id_1',
        1596: 'id_1',
        1605: 'id_1',
        1614: 'id_1',
        1623: 'id_1',
        1632: 'id_1',
        1641: 'id_1',
        1650: 'id_1',
        1659: 'id_1',
        1668: 'id_1',
        1677: 'id_1',
        1686: 'id_1',
        1695: 'id_1',
        1704: 'id_1',
        1713: 'id_1',
        1722: 'id_1',
        1731: 'id_1',
        1740: 'id_1',
        1749: 'id_1',
        1758: 'id_1',
        1767: 'id_1',
        1776: 'id_1',
        1785: 'id_1',
        1794: 'id_1',
        1795: 'id_8',
        1804: 'id_1',
        1805: 'id_8',
        1814: 'id_1',
        1815: 'id_8',
        18

In [46]:
import json

metrics = {'RandomForestRegressor': 37.03840070678769, 'XGBRegressor': 37.25298017954496, 'LGBMRegressor': 38.51687419786843}       

In [47]:
for item in metrics:
    print(item)

RandomForestRegressor
XGBRegressor
LGBMRegressor


In [53]:
for key, value in metrics.items():
    print(f"SMAPE of {key}: {value}") 

SMAPE of RandomForestRegressor: 37.03840070678769
SMAPE of XGBRegressor: 37.25298017954496
SMAPE of LGBMRegressor: 38.51687419786843
